In [1]:
! python --version

Python 3.8.20


In [2]:
! pip install datasets transformers rouge-score jieba

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e7939d9a4f5bc06a6d0ac6d8573d5790d932288434a79bb3980ded8419b14b53
  Stored in directory: /Users/wuyinghui/Library/Caches/pip/wheels/88/8d/5a/cbcddcc5d2fae2c2f8d83a91c3f5205afee148935880a9c29b
Successfully built rouge-score


In [3]:
! pip install --upgrade ipywidgets tqdm
! jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [1]:
from ipywidgets import IntProgress
import tqdm 
from datasets import load_dataset
dataset = load_dataset('json', data_files='nlpcc_data.json', field='data')

In [2]:
dataset["train"]

Dataset({
    features: ['title', 'content'],
    num_rows: 50000
})

In [3]:
def flatten(example):
    return {
        "document": example["content"],
        "summary": example["title"],
        "id":"0"
    }
dataset = dataset["train"].map(flatten, remove_columns=["title", "content"]) # , remove_columns=["title", "content"]


In [4]:
TokenModel = "bert-base-chinese"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(TokenModel)

In [5]:
model_checkpoint = "facebook/bart-base-cnn"
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = "" # BART-12-3

In [6]:
max_input_length = 1024 # input, source text
max_target_length = 256 # summary, target text

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [7]:
dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 50000
})

In [8]:
raw_datasets = dataset
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [9]:
from datasets import dataset_dict
import datasets

train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()
train_data_txt, test_data_tex = train_data_txt.train_test_split(test_size=0.1).values()
# 装载数据
dd = datasets.DatasetDict({"train":train_data_txt,"validation": validation_data_txt,"test":test_data_tex }) 

raw_datasets = dd
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)


Map:   0%|          | 0/40500 [00:00<?, ? examples/s]

/Users/wuyinghui/opt/anaconda3/envs/llm/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

In [10]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 40500
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 4500
    })
})

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(raw_datasets["train"])

,document,summary,id
0,"菲律宾总统贝尼尼奥·阿基诺三世访问日本期间就南海问题大放厥词,言辞充满挑衅,引发中方强烈不满。这一次,连菲律宾本国媒体都看不下去了,厉声告诫阿基诺找回理智,不要玩火自焚。【莫要玩火,小心自焚】新华国际客户端注意到,菲律宾《马尼拉时报》4日发表社论,题为《总统对中国发起不必要的挑衅》。文章说,阿基诺访日期间发表演讲,提及争议领土问题(指南海问题),再次搅动时局。文章说:“这种对中国的公然挑衅看起来毫无必要,也不清楚他到底希望借此维护什么国家利益。”日本媒体报道,阿基诺把中国在南海的行为与二战前的纳粹德国相提并论,同时呼吁美国作为超级大国在南海问题上发挥作用。社论说,菲律宾政府的行为实际上令南海局势趋于恶化。美国增加了在这一地区的海军存在,对中国的言辞也更加激烈。“我们只盼望阿基诺总统心里有谱,能理智地终结这个局面。难道按照他的期望,美国海军在争议领土附近的存在能吓走中国?”社论问道。文章说,中国反而有可能强化立场,“我们怀疑菲律宾是否已经准备好应对那样的局面”。社论告诫阿基诺政府不要玩火自焚。“难道菲律宾期望中国和美国围绕争议领土爆发一场热战?我们希望不会,因为当两个巨人碰撞时,小个子如菲律宾可能会被压扁。”更重要的是,亚太地区的不稳定将威胁到整个地区的经济发展,继而殃及菲律宾。社论说,菲律宾与中国的关系不应由领土争端“界定”。相比对中国采取不必要的对抗、致使中美关系紧张,与中国保持友好关系,菲律宾反而能够获得更多。“我们相信能够经由外交手段和平化解冲突。为了与中国达成和平互利的解决方案,阿基诺总统应该记住一句老话,‘用蜂蜜捉苍蝇,总比用醋强’。”【丢掉幻想,回头是岸】针对阿基诺关于南海问题的言论,外交部发言人华春莹3日在例行记者会上表示,中方对菲方领导人的“荒谬无理言论”深感震惊并强烈不满。“我再次严肃正告菲方某些人丢掉幻想,回头是岸,停止挑拨挑衅,回到通过双边渠道谈判协商解决争议的正确轨道上来。”华春莹说。华春莹说,回顾一下南海争议的事实经纬不难发现,上世纪70年代以来武力非法侵占中国南沙群岛部分岛礁的是菲律宾;1999年以来企图以“坐滩”形式窃占中国仁爱礁的是菲律宾;2012年派遣军舰武力袭扰在中国黄岩岛海域正常作业的中国渔船、渔民的是菲律宾;2013年无视中方作为《联合国海洋法公约》缔约国应有的权利、违背《南海各方行为宣言》和两国间一系列共识,单方面将有关争议提交所谓国际仲裁的是菲律宾;近年来出于一己私利不断勾结域外国家搅浑水、抹黑攻击中国的还是菲律宾。她表示,中国是重信守诺的负责任国家,始终致力于通过同直接当事国之间的谈判协商解决有关争议,同时,中国政府维护领土主权和海洋权益的意志“坚定不移”。(记者杨天沐,编辑韩梁、胡若愚,新华国际客户端独家报道)","阿基诺访日时演讲提及南海问题,菲媒称其对中国公然挑衅毫无必要,告诫其找回理智,不要玩火自焚",0
1,"本报讯(记者党文民)7月26日14时,省气象台发布干旱橙色预警,预计未来7天我省无大范围降水天气过程,干旱将进一步发展,请干旱地区做好防范。根据综合气象干旱指数监测,截至7月25日,全省有47%的县市气象干旱达到重旱等级,有17%的县市达到特旱等级。重旱以上的区域包括郑州、开封、漯河、平顶山、周口、焦作、商丘、许昌8个省辖市和三门峡、洛阳、南阳、驻马店、新乡5个省辖市的部分区域,其中特旱区分布在开封、许昌、平顶山、周口、焦作。今年6月份以来,我省平均降水量比往年同期大幅减少。省气象局气候中心高级工程师张善强说,这是由于今年的南海季风爆发时间偏晚,携带水汽偏少,而控制我省的副热带高压今年的位置偏南偏西,也阻断了水汽向北输送,加上冷空气势力较弱,因此在我省形成有效降水的几率大大降低。据了解,干旱预警信号分为橙色、红色两个级别。橙色预警的标准为:预计未来一周综合气象干旱指数达到重旱(气象干旱为25~50年一遇),或者某一县(区)有40%以上的农作物受旱。此时,有关部门应及时采取启用应急备用水源,压减城镇供水指标,限制非生产性高耗水及服务业用水等防御措施。","河南发布干旱橙色预警,未来7天无大范围降水,全省47%县市达到重旱等级。",0
2,"昨天,在浙江宁波一家超市,因其没有进问题批次的货,所以还在正常销售雅培幼儿喜康力(3段)900克/罐装奶粉。图/CFP昨天,国家质检总局表示,我国已对恒天然集团浓缩乳清蛋白粉和奶粉基粉两种原料无限期停止进口,直到污染事件完全解决。恒天然方面则确认雅培为事先要求不公布的企业。另外,新西兰总理表示,或将考虑为“毒奶粉”事件赴华。□说法■恒天然雅培为最后一家牵连企业据恒天然的公关公司提供的5日中午发布会的现场速记称,此次涉事的38吨浓缩乳清蛋白粉中有18吨为恒天然自己分别在澳大利亚和新西兰的工厂使用,最主要用来生产基粉,用于生产婴幼儿营养品。其中,澳大利亚工厂生产的产品涉及到两家生产婴儿营养制品的公司,除了已公开的拥有多美滋、可瑞康等品牌的达能外,另外一家公司则暂时不让恒天然提及它们的名字。那么这家不愿意提及名称的企业是否就是雅培?对此,雅培方面昨天凌晨接受本报采访时给予了否认。其表示,首先雅培并未使用可能受到污染的原料,另外,雅培产品是在新西兰实施包装的,而不是在澳大利亚包装的。不过,昨天晚间,恒天然有关负责人郝晓红则向记者表示,这家不愿意提及名称的企业就是雅培,雅培的产品原料确实未涉及到这38吨问题浓缩乳清蛋白粉,只是受到了生产线的影响。澄清两天内是“启动召回”8月5日中午,恒天然首席执行官西奥·史毕根斯在北京的媒体发布会上表示:恒天然未来48小时内相关产品将会得到召回。对此说法,有消息引用涉事企业的表态称“这是不可能完成的任务”。而按照“48小时内相关产品得到召回”的说法,就意味着今天中午将是召回时间的最后时限。不过对于这种说法,恒天然方面有关负责人郝晓红昨天接受记者采访时则澄清表示:发布会现场可能存在翻译和速记的误解,48小时内指的是“启动召回和进行召回措施”,而非“完成召回”。至于何时完成召回,则需要一个过程和时间。郝晓红还表示,到8月6日凌晨雅培公布防御性召回措施时,所有涉事的企业名单得到了公布,其实在24小时内就全面启动了对相关产品的召回。■雅培涉事产品北京极少量有售在恒天然发布会结束后,昨天凌晨,此前一直表示自己的原料并未涉及到恒天然这批问题浓缩乳清蛋白粉的雅培也宣布,对两批次幼儿喜康力(3段)奶粉进行“预防性回收”并销毁,召回产品共7181箱,其中约112箱已售出。召回范围内的产品批号分别为287834K402和287844K402,生产日期均为2013年5月2日,失效日期为2015年10月31日。雅培特别强调,雅培供应中国市场的所有产品均未使用恒天然受污染的乳清蛋白粉,但经恒天然方面8月4日晚间确认,上述2个批次供应中国市场的雅培金装幼儿喜康力听装奶粉在恒天然公司包装线上实施包装,但该包装生产线在使用有问题原料后未经彻底清洗即开始包装雅培产品。雅培方面昨天表示,目前无法提供涉事产品完全准确的销售地点,但确实有进入北京市场,不过数量很少,大约为5箱以内。■多美滋新增加两个召回奶粉批次卷入恒天然涉毒浓缩乳清蛋白粉的多美滋昨天也再次发声明称,召回的产品由此前的12个批次增加到14个批次。多美滋声明显示,新增召回的2个奶粉批次为多领加延续较大婴幼儿配方奶粉(2阶段)(2013年5月23日之后生产)1200g,批号分别为1F3199、1F3208。与此同时,多美滋还更改了一个批次产品的相关信息,此前公布召回的优阶贝护延续较大婴儿配方2阶段奶粉850g+50g的1H3172,改成900g产品的1H3172。多美滋方面表示,新增的两个批次是根据恒天然8月5日的新增信息,这2个批次的多领加2阶段产品有可能受到影响。多美滋启动预防性召回。对此,多美滋方面称,因为此前1H3172批次的净重被混淆,调整是为了纠正此前的工作失误。此前的8月4日,多美滋表示,已查明部分优阶贝护和多领加2阶段产品有可能受到影响,共涉及12个批次。其中流入市场的问题多美滋奶粉有420.188吨。如今新增两批次产品,多美滋方面昨天晚间接受记者采访时确认,相关产品有流入市场,但具体的量暂时还没有统计数据,届时以相关部门公布数据为准。不过公司已正采取措施迅速下架这两个批次的产品。□发布■国家质检总局无限期停进口涉事两原料昨天上午,国家质检总局发出通告,要求雅培公司召回两个可能被污染的批次奶粉。另外,目前我国已对恒天然集团浓缩乳清蛋白粉和奶粉基粉两种原料无限期停止进口,直到污染事件完全解决。目前,要根据受污染的乳清蛋白粉的范围和市场流向的掌握情况以及恒天然集团的报告来决定何时恢复进口。■国家食品药品监督管理总局生产婴幼儿配方乳粉企业生产条件审查细则征意见为方便广大消费者、食品监管部门和社会各界查询信息,鼓励广大消费者和社会各界参与婴幼儿配方乳粉质量安全监督,国家食品药品监督管理总局6日对外公布各省级食品监管部门批准取得食品生产许可证的128家婴幼儿配方乳粉生产企业的相关信息。这128家企业包括美赞臣营养品(中国)有限公司、多美滋婴幼儿食品有限公司、雅培(广州)营养品有限公司、内蒙古伊利实业集团股份有限公司等,具体名单可在国家食品药品监督管理总局网站查询。此外,由国家食品药品监管总局组织专家、食品安全监督部门人员研究、修订的《企业生产婴幼儿配方乳粉许可条件审查细则(2013版)(征求意见稿)》,现面向全社会公开征求意见。据了解,新的细则征求意见稿参照药品管理全面提高了婴幼儿配方乳粉生产企业生产、管理要求。□表态新西兰总理考虑赴华为应对乳制品巨头恒天

In [1]:
! pip install transformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


In [1]:
! pip3 install fire

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
  Using cached fire-0.7.0-py3-none-any.whl


In [2]:
  
import warnings
from pathlib import Path
from typing import List, Tuple, Union

import fire
from torch import nn

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedModel
from transformers.utils import logging

logger = logging.get_logger(__name__)

In [3]:
def copy_layers(src_layers: nn.ModuleList, dest_layers: nn.ModuleList, layers_to_copy: List[int]) -> None:
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())


LAYERS_TO_COPY = {
    # maps  num layers in teacher -> num_layers in student -> which teacher layers to copy.
    # 12: bart, 16: pegasus, 6: marian/Helsinki-NLP
    12: {
        1: [0],  # This says that if the teacher has 12 layers and the student has 1, copy layer 0 of the teacher
        2: [0, 6],
        3: [0, 6, 11],      # the first, 7th and 12th decode layers
        4: [0, 4, 8, 11],
        6: [0, 2, 4, 7, 9, 11],
        9: [0, 1, 2, 4, 5, 7, 9, 10, 11],
        12: list(range(12)),
    },
    16: {  # maps  num layers in student -> which teacher layers to copy
        1: [0],
        2: [0, 15],
        3: [0, 8, 15], 
        4: [0, 5, 10, 15],
        6: [0, 3, 6, 9, 12, 15],
        8: [0, 2, 4, 6, 8, 10, 12, 15],
        9: [0, 1, 3, 5, 7, 9, 11, 13, 15],
        12: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15],
        16: list(range(16)),
    },
    6: {1: [0], 2: [0, 5], 3: [0, 2, 5], 4: [0, 1, 3, 5], 6: list(range(6))},
}
LAYERS_TO_SUPERVISE = {
    # maps  num layers in student -> which teacher layers to copy.
    6: {1: [5], 2: [3, 5], 3: [1, 4, 5], 4: [1, 2, 4, 5]},
    12: {1: [11], 2: [5, 11], 3: [3, 7, 11], 6: [1, 3, 5, 8, 10, 11]},
    16: {1: [15], 4: [4, 9, 12, 15], 8: [1, 3, 5, 7, 9, 11, 13, 15]},
}


In [4]:
def create_student_by_copying_alternating_layers(
    teacher: Union[str, PreTrainedModel],
    save_path: Union[str, Path] = "student",
    e: Union[int, None] = None,
    d: Union[int, None] = None,
    copy_first_teacher_layers=False,
    e_layers_to_copy=None,
    d_layers_to_copy=None,
    **extra_config_kwargs
) -> Tuple[PreTrainedModel, List[int], List[int]]:
    
    _msg = "encoder_layers and decoder_layers cannot be both None-- you would just have an identical teacher."
    assert (e is not None) or (d is not None), _msg
    if isinstance(teacher, str):
        AutoTokenizer.from_pretrained(teacher).save_pretrained(save_path)  # purely for convenience
        teacher = AutoModelForSeq2SeqLM.from_pretrained(teacher).eval()
    else:

        assert isinstance(teacher, PreTrainedModel), f"teacher must be a model or string got type {type(teacher)}"
    init_kwargs = teacher.config.to_diff_dict()

    try:
        teacher_e, teacher_d = teacher.config.encoder_layers, teacher.config.decoder_layers
        if e is None:
            e = teacher_e
        if d is None:
            d = teacher_d
        init_kwargs.update({"encoder_layers": e, "decoder_layers": d})
    except AttributeError:  # T5
        teacher_e, teacher_d = teacher.config.num_layers, teacher.config.num_decoder_layers
        if e is None:
            e = teacher_e
        if d is None:
            d = teacher_d
        init_kwargs.update({"num_layers": e, "num_decoder_layers": d})

    # Kwargs to instantiate student: teacher kwargs with updated layer numbers + **extra_config_kwargs
    init_kwargs.update(extra_config_kwargs)

    # Copy weights
    student_cfg = teacher.config_class(**init_kwargs)
    student = AutoModelForSeq2SeqLM.from_config(student_cfg)
    # Start by copying the full teacher state dict this will copy the first N teacher layers to the student.
    info = student.load_state_dict(teacher.state_dict(), strict=False)
    assert info.missing_keys == [], info.missing_keys  # every student key should have a teacher keys.

    if copy_first_teacher_layers:  # Our copying is done. We just log and save
        e_layers_to_copy, d_layers_to_copy = list(range(e)), list(range(d))
        logger.info(
            f"Copied encoder layers {e_layers_to_copy} and decoder layers {d_layers_to_copy}. Saving them to {save_path}"
        )
        student.save_pretrained(save_path)
        return student, e_layers_to_copy, d_layers_to_copy

    # Decide which layers of the teacher to copy. Not exactly alternating -- we try to keep first and last layer.
    if e_layers_to_copy is None:
        e_layers_to_copy: List[int] = pick_layers_to_copy(e, teacher_e)
    if d_layers_to_copy is None:
        d_layers_to_copy: List[int] = pick_layers_to_copy(d, teacher_d)

    try:
        copy_layers(teacher.model.encoder.layers, student.model.encoder.layers, e_layers_to_copy)
        copy_layers(teacher.model.decoder.layers, student.model.decoder.layers, d_layers_to_copy)
    except AttributeError:  # For t5, student.model.encoder.layers is called student.encoder.block
        copy_layers(teacher.encoder.block, student.encoder.block, e_layers_to_copy)
        copy_layers(teacher.decoder.block, student.decoder.block, d_layers_to_copy)
    logger.info(
        f"Copied encoder layers {e_layers_to_copy} and decoder layers {d_layers_to_copy}. Saving them to {save_path}"
    )
    student.config.init_metadata = dict(
        teacher_type=teacher.config.model_type,
        copied_encoder_layers=e_layers_to_copy,
        copied_decoder_layers=d_layers_to_copy,
    )
    student.save_pretrained(save_path)
    # Save information about copying for easier reproducibility

    return student, e_layers_to_copy, d_layers_to_copy


In [5]:
def pick_layers_to_copy(n_student, n_teacher):
    try:
        val = LAYERS_TO_COPY[n_teacher][n_student]
        return val
    except KeyError:
        if n_student != n_teacher:
            warnings.warn(
                f"no hardcoded layers to copy for teacher {n_teacher} -> student {n_student}, defaulting to first {n_student}"
            )
        return list(range(n_student))


In [6]:
model, list_en, list_de = create_student_by_copying_alternating_layers(model, 'trian.pth', 12, 3)

NameError: name 'model' is not defined

In [26]:
batch_size = 2
args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=batch_size,  # demo
    per_device_eval_batch_size=batch_size,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)


In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [28]:
import jieba
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(jieba.cut(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(jieba.cut(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}


In [29]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
! pip install torch==1.5.0

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 752.0 MB 14.5 MB/s eta 0:00:01MB 68.6 MB/s eta 0:00:11 MB 68.6 MB/s eta 0:00:11
  Attempting uninstall: torch
    Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.5.0 requires torch==1.4.0, but you have torch 1.5.0 which is incompatible.
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.4/bin/python -m pip install --upgrade pip' command.


In [55]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, token_type_ids, document, summary.
***** Running training *****
  Num examples = 40500
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 20250


Step,Training Loss
50,12.439500
100,7.493300
150,6.301900
200,6.089800
250,6.076500
300,5.507800
350,4.921700
400,5.299200
450,5.257500
500,5.245900


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json


Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in results/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to results/checkpoint-2000
Configuration saved in results/checkpoint-2000/config.json
Model weights saved in results/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-2000/tokenizer_config.json
Special tokens file saved in results/checkpoint-2000/special_tokens_map.json
Deleting olde

TrainOutput(global_step=20250, training_loss=3.7772197069709685, metrics={'train_runtime': 5608.721, 'train_samples_per_second': 7.221, 'train_steps_per_second': 3.61, 'total_flos': 5.519068250185728e+16, 'train_loss': 3.7772197069709685, 'epoch': 1.0})

In [57]:
import torch

In [58]:
torch.save(model.state_dict(), "BART.pth")

In [32]:
import torch
model.load_state_dict(torch.load('BART.pth'))

<All keys matched successfully>

In [33]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


In [34]:
test_samples = "2005年6月，习近平同志首次提出“红船精神”，将其概括为“开天辟地、敢为人先的首创精神，坚定理想、百折不挠的奋斗精神，立党为公、忠诚为民的奉献精神”，深刻阐述了“红船精神”的丰富内涵、历史地位、时代价值。2017年10月，党的十九大闭幕仅一周，习近平总书记就带领中共中央政治局常委同志，瞻仰上海中共一大会址和浙江嘉兴南湖红船，回顾建党历史，重温入党誓词。习近平总书记在南湖革命纪念馆参观时指出：“我们要结合时代特点大力弘扬‘红船精神’。”“红船精神”是中国革命精神之源，激励着我们党砥砺前行、发展壮大，是我们党立党兴党、执政兴国的宝贵精神财富。"

In [35]:
a = generate_summary(test_samples, model)

In [36]:
a[1][0]

'[unused2] 月 睾 习 近 平 同 志 首 次 提 出 红 船 精 神 睾 将 其 概 括 为 开 天 辟 地 、 敢 为 人 先 的 首 创 精 神 睾 立 党 为 公 、 忠 诚 为 民 的 奉 献 精 神 。... 。 现 场 图 ) 组 图 ( 组 图 : ) 图 图 ).. 6 图 : 习 6. 。 组 组 图 ) 组 图 图 ) 。 图 图 组 原 金 金 重 。 现 现 场 上 海 [unused2]'

In [73]:
list(raw_datasets["test"]["summary"][0:100])

['重庆:妈妈半斤白酒下肚后喂奶灌醉宝宝,孩子吃完奶皮肤发烫、手脚乱动,吐奶后清醒过来。',
 '许昌住院女子与男护士发生关系 结婚后不如意状告院方',
 '资阳原市委书记李佳涉嫌受贿罪被逮捕,案件侦查工作正在进行中。',
 '遵义气象台20时55分发布雷电黄色预警:目前遵义县、桐梓等地出现雷雨天气,预计未来将影响中东部地区。',
 '海南藏族自治州发布暴雨橙色预警:预计共和地区3小时内降雨量将达25毫米以上且降雨量可能持续,引发洪涝、泥石流、城...',
 '昆明一母亲买菜留孩子一人在家,2岁半娃娃4楼坠下气息奄奄已不能哭闹说话;孩子目前已送医仍在抢救。',
 '厦门23岁女生去福州看同学,搭乘出租车后失踪,昨晚该女生手机始终占线',
 '长江翻船浙江籍乘客儿女:爸妈,求你们再更新一次朋友圈;11名乘客家属到达监利,盼看老人最后一眼',
 '酒泉市发布道路结冰黄色预警:预计未来72小时内,我市肃州区仍有道路结冰,请注意防范。...',
 '国资委:唐复平不再担任鞍钢总经理,杨华不再担任鞍钢党委书记等,姚林兼任鞍钢党委书记和总经理。',
 '新疆边境一线反恐战斗现场曝光:一伙暴徒准备逃往国外参加“圣战”,武警获取情报主动出击,经激烈枪战歼灭6名暴徒。',
 '歌手陈红遭前夫起诉,被指以军人身份经商;前夫曾将名下9家公司股权半数转让给陈红,现要求收回。 ',
 '菲考虑解雇16名中国专家 ,称为确保“国家电网安全”,应改聘菲律宾人;报道称中国专家须本周离开。',
 '外交部发言人:中国赴菲律宾第一批救援人员将于20日启程',
 '南京给试点汽车尾气管"戴口罩" PM2.5浓度降九成',
 '西甲:梅西内马尔传射,巴萨3-0完胜埃瓦尔,4分优势领跑,下轮将客场迎战皇马。',
 '厦门:男子扬言携炸弹上公交车 惊动排爆特警',
 '覃塘区平天山阴坑一处山体因暴雨发生坍塌,致山脚下一工棚被冲垮4人被埋,其中2人获救,另2人失踪。',
 '招远血案被害人家属至今未获赔偿,生活陷困顿;多次咨询政府未获实质性说法,称觉得自己已成“乞丐”,整天去政府要钱',
 '延安:“被逼卖处”嫌犯家属否认央求官员帮忙调解,称“封口费与我无关”;当地政府承认未向当事人了解情况',
 '侯树森不再担任解放军副总参谋长职务,空军将领乙晓光接替其工作',
 '<教养>《魔法亲亲》、《一口袋的吻》……这

In [63]:
pred_list = []
gold_list = []
for text in raw_datasets["test"]["document"][0:100]:
    pred_list.append(generate_summary(text, model)[1][0])
gold_list = list(raw_datasets["test"]["summary"][0:100])

In [41]:
! pip install lawrouge

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
  Created wheel for lawrouge: filename=lawrouge-2.0.0-py3-none-any.whl size=9289 sha256=de9df0aaaabcba163b251af9c1ecfb236f58debf9d1b470430f74b09822f5daf
  Stored in directory: /home/ma-user/.cache/pip/wheels/89/d6/bd/fafe523360e3233e6a58600caf802538dc04171fb32423f665
Successfully built lawrouge
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.4/bin/python -m pip install --upgrade pip' command.


In [53]:
import lawrouge
rouge = lawrouge.Rouge()
score = rouge.get_scores(["他是清华大学计算机科学与技术系。计算机科学与技术专业。"], ["他是清华大学计算机科学与技术系。"], avg=0)
score[0]["rouge-2"]["r"]

1.0

In [ ]:
pred_list

In [80]:
import lawrouge
rouge = lawrouge.Rouge()
score_s = 0
for i in range(len(gold_list)):
    score = rouge.get_scores([pred_list[i].replace(" ", "")], [gold_list[i].replace(" ", "")], avg=0)
    score_s += score[0]["rouge-l"]["r"]
score_ave = score_s/len(gold_list)
print('weighted score: ', score_ave)

weighted score:  0.45293061867921885


In [74]:
pred_list[1].replace(" ", )

'[unused2][夜宵]章子怡示爱汪峰被恶搞爆粗口回击:扯nmb;谢娜当众掐黄磊女儿脖子被批:没素质没爱心;周杰伦女友昆凌:看新闻才知婚讯。女友:我也是看了新闻...(组图)。))知情情案。详细也是新闻:看了看了)。看了。看。组图合同[unused2]'

In [86]:
rouge = lawrouge.Rouge()
scores = rouge.get_scores(["他是清华大学计算机科学与技术系"], ["计算机科学与技术专业"], avg=2)
print(scores)

{'f': 0.6274782560840833, 'p': 0.52, 'r': 0.7911111111111112}


In [ ]:
raw_datasets["test"]["summary"]

In [ ]:
import moxing as mox
mox.file.copy('/home/ma-user/work/obs_file.txt', 'obs://bart-summ/obs_file.txt')